In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')


In [2]:
DATASET_DIRECTORY = 'C:/Users/EDWIN/Downloads/CICIoT2023'
df_sets = [k for k in os.listdir(DATASET_DIRECTORY) if k.endswith('.csv')]
df_sets.sort()
dfs=[]
#Dropping NA values
for file in tqdm(df_sets):
    file_path=os.path.join(DATASET_DIRECTORY,file)
    df=pd.read_csv(file_path)
    df.dropna()
    dfs.append(df)
    
print(dfs[0])



100%|██████████| 169/169 [02:10<00:00,  1.29it/s]


        flow_duration  Header_Length  Protocol Type  Duration         Rate  \
0            0.000000          54.00           6.00     64.00     0.329807   
1            0.000000          57.04           6.33     64.00     4.290556   
2            0.000000           0.00           1.00     64.00    33.396799   
3            0.328175       76175.00          17.00     64.00  4642.133010   
4            0.117320         101.73           6.11     65.91     6.202211   
...               ...            ...            ...       ...          ...   
238682       0.000000          54.00           6.00     64.00     3.049186   
238683       0.000000          54.00           6.00     64.00   183.433732   
238684       0.000785          56.29           6.11     64.00   306.952216   
238685       0.000901          72.09           6.11     64.64   158.475986   
238686       0.000000           0.00           1.00     64.00     1.291274   

              Srate  Drate  fin_flag_number  syn_flag_number  r

In [3]:
def map_to_binary(label):
    if label == 'BenignTraffic':
        return 0  # Normal class
    else:
        return 1  # Attack classes

for df in tqdm(dfs):
    df['label'] = df['label'].apply(map_to_binary)


for df in dfs:
    print(df['label'].value_counts())


100%|██████████| 169/169 [00:10<00:00, 16.29it/s]


label
1    233087
0      5600
Name: count, dtype: int64
label
1    213605
0      5200
Name: count, dtype: int64
label
1    268871
0      6387
Name: count, dtype: int64
label
1    225726
0      5297
Name: count, dtype: int64
label
1    222105
0      5386
Name: count, dtype: int64
label
1    234259
0      5787
Name: count, dtype: int64
label
1    228256
0      5537
Name: count, dtype: int64
label
1    222583
0      5327
Name: count, dtype: int64
label
1    240453
0      5874
Name: count, dtype: int64
label
1    222152
0      5464
Name: count, dtype: int64
label
1    245400
0      5992
Name: count, dtype: int64
label
1    216707
0      5221
Name: count, dtype: int64
label
1    233006
0      5470
Name: count, dtype: int64
label
1    233081
0      5680
Name: count, dtype: int64
label
1    210792
0      5046
Name: count, dtype: int64
label
1    262222
0      6427
Name: count, dtype: int64
label
1    227485
0      5400
Name: count, dtype: int64
label
1    218245
0      5199
Name: count, dtype

In [11]:
for idx,df in tqdm(enumerate(dfs)):
    corr=df.corr()
    columns = np.full((corr.shape[0],), True, dtype=bool)
    for i in range(corr.shape[0]):
        for j in range(i+1, corr.shape[0]):
            if corr.iloc[i,j] >= 0.9:
                if columns[j]:
                    columns[j] = False
    selected_columns = df.columns[columns]
    selected_columns.shape
    dfs[idx]=df[selected_columns]

for df in dfs:
    print(df.shape)



169it [02:47,  1.01it/s]

(238687, 35)
(218805, 36)
(275258, 35)
(231023, 35)
(227491, 34)
(240046, 35)
(233793, 36)
(227910, 36)
(246327, 35)
(227616, 35)
(251392, 35)
(221928, 35)
(238476, 36)
(238761, 36)
(215838, 36)
(268649, 35)
(232885, 36)
(223444, 36)
(234430, 36)
(231063, 35)
(237151, 36)
(223401, 35)
(252212, 35)
(262062, 35)
(225725, 36)
(447625, 35)
(236152, 35)
(221014, 36)
(251225, 36)
(251676, 35)
(238350, 35)
(228730, 36)
(227636, 35)
(447774, 35)
(232228, 36)
(252744, 36)
(439212, 34)
(214963, 36)
(236660, 36)
(437362, 35)
(230365, 34)
(445281, 36)
(222425, 36)
(250954, 36)
(231980, 35)
(247290, 35)
(235566, 35)
(239086, 35)
(219877, 35)
(248997, 35)
(233625, 36)
(251256, 36)
(445425, 35)
(243279, 35)
(235625, 35)
(222384, 35)
(235026, 35)
(429585, 36)
(236995, 36)
(245184, 35)
(254147, 35)
(239643, 36)
(441027, 35)
(237426, 36)
(263065, 35)
(244926, 36)
(216084, 35)
(267928, 35)
(224089, 35)
(240206, 35)
(229776, 36)
(277720, 34)
(260695, 35)
(235721, 34)
(247037, 36)
(444334, 36)
(446795, 35)

In [13]:
# Step 1: Get the intersection of selected columns (common across all dataframes)
common_columns = set(dfs[0].columns)  # Start with the columns from the first dataframe

for df in dfs[1:]:
    common_columns = common_columns.intersection(set(df.columns))

common_columns = list(common_columns)

# Step 2: Reindex all dataframes to use only the common columns
for idx, df in tqdm(enumerate(dfs)):
    dfs[idx] = df[common_columns]

# Now all dfs have the same number of columns (common columns), so you can fit the model


169it [00:27,  6.06it/s]


In [14]:
for df in dfs:
    print(df.shape)

(238687, 34)
(218805, 34)
(275258, 34)
(231023, 34)
(227491, 34)
(240046, 34)
(233793, 34)
(227910, 34)
(246327, 34)
(227616, 34)
(251392, 34)
(221928, 34)
(238476, 34)
(238761, 34)
(215838, 34)
(268649, 34)
(232885, 34)
(223444, 34)
(234430, 34)
(231063, 34)
(237151, 34)
(223401, 34)
(252212, 34)
(262062, 34)
(225725, 34)
(447625, 34)
(236152, 34)
(221014, 34)
(251225, 34)
(251676, 34)
(238350, 34)
(228730, 34)
(227636, 34)
(447774, 34)
(232228, 34)
(252744, 34)
(439212, 34)
(214963, 34)
(236660, 34)
(437362, 34)
(230365, 34)
(445281, 34)
(222425, 34)
(250954, 34)
(231980, 34)
(247290, 34)
(235566, 34)
(239086, 34)
(219877, 34)
(248997, 34)
(233625, 34)
(251256, 34)
(445425, 34)
(243279, 34)
(235625, 34)
(222384, 34)
(235026, 34)
(429585, 34)
(236995, 34)
(245184, 34)
(254147, 34)
(239643, 34)
(441027, 34)
(237426, 34)
(263065, 34)
(244926, 34)
(216084, 34)
(267928, 34)
(224089, 34)
(240206, 34)
(229776, 34)
(277720, 34)
(260695, 34)
(235721, 34)
(247037, 34)
(444334, 34)
(446795, 34)

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score


timesteps=1
# Build LSTM model
model = Sequential()
model.add(LSTM(128, input_shape=(timesteps, dfs[0].shape[1]), activation='relu'))
model.add(Dense(64, activation='relu'))  
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

scaler = MinMaxScaler()
y_true = []
y_pred = []

for df in tqdm(dfs):
    y=df['label']
    data = df.values
    data_reshaped = data.reshape((data.shape[0], timesteps, data.shape[1]))
    X_train, X_test, y_train, y_test = train_test_split(data_reshaped, y, test_size=0.2, random_state=42)

    y_true += list(y_test)

    # Train the model
    history = model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.2)

    y_pred_probs = model.predict(X_test)
    y_pred += list((y_pred_probs > 0.5).astype(int))


  0%|          | 0/169 [00:00<?, ?it/s]

Epoch 1/5
4774/4774 ━━━━━━━━━━━━━━━━━━━━ 8s 1ms/step - accuracy: 0.9542 - loss: 8491.7871 - val_accuracy: 0.9802 - val_loss: 91.6492
Epoch 2/5
4774/4774 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - accuracy: 0.9736 - loss: 24.9190 - val_accuracy: 0.9760 - val_loss: 134.7767
Epoch 3/5
4774/4774 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - accuracy: 0.9679 - loss: 13.6657 - val_accuracy: 0.9762 - val_loss: 0.0976
Epoch 4/5
4774/4774 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - accuracy: 0.9769 - loss: 0.0992 - val_accuracy: 0.9760 - val_loss: 0.0848
Epoch 5/5
4774/4774 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - accuracy: 0.9772 - loss: 0.1131 - val_accuracy: 0.9760 - val_loss: 0.0798
1492/1492 ━━━━━━━━━━━━━━━━━━━━ 1s 650us/step


  1%|          | 1/169 [00:33<1:33:03, 33.24s/it]

Epoch 1/5
4377/4377 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - accuracy: 0.9764 - loss: 0.1422 - val_accuracy: 0.9769 - val_loss: 0.0913
Epoch 2/5
4377/4377 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - accuracy: 0.9796 - loss: 1.3353 - val_accuracy: 0.9803 - val_loss: 0.0907
Epoch 3/5
4377/4377 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - accuracy: 0.9793 - loss: 0.1031 - val_accuracy: 0.9803 - val_loss: 0.0906
Epoch 4/5
4377/4377 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.9802 - loss: 0.0813 - val_accuracy: 0.9803 - val_loss: 0.0862
Epoch 5/5
4377/4377 ━━━━━━━━━━━━━━━━━━━━ 5s 1ms/step - accuracy: 0.9805 - loss: 0.0780 - val_accuracy: 0.9804 - val_loss: 0.0829
1368/1368 ━━━━━━━━━━━━━━━━━━━━ 1s 595us/step


  1%|          | 2/169 [01:02<1:25:33, 30.74s/it]

...

 99%|█████████▉| 167/169 [1:43:40<01:47, 53.54s/it]

Epoch 1/5
4554/4554 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - accuracy: 0.9799 - loss: 0.3765 - val_accuracy: 0.9804 - val_loss: 0.0760
Epoch 2/5
4554/4554 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - accuracy: 0.9803 - loss: 0.0766 - val_accuracy: 0.9804 - val_loss: 0.0759
Epoch 3/5
4554/4554 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - accuracy: 0.9803 - loss: 0.0762 - val_accuracy: 0.9805 - val_loss: 0.0758
Epoch 4/5
4554/4554 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - accuracy: 0.9803 - loss: 0.0773 - val_accuracy: 0.9805 - val_loss: 0.0764
Epoch 5/5
4554/4554 ━━━━━━━━━━━━━━━━━━━━ 7s 1ms/step - accuracy: 0.9805 - loss: 0.0762 - val_accuracy: 0.9804 - val_loss: 0.0759
1424/1424 ━━━━━━━━━━━━━━━━━━━━ 1s 644us/step


 99%|█████████▉| 168/169 [1:44:12<00:47, 47.01s/it]

Epoch 1/5
4695/4695 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - accuracy: 0.9809 - loss: 0.0887 - val_accuracy: 0.9813 - val_loss: 0.0758
Epoch 2/5
4695/4695 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - accuracy: 0.9806 - loss: 0.0766 - val_accuracy: 0.9813 - val_loss: 0.0758
Epoch 3/5
4695/4695 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - accuracy: 0.9801 - loss: 0.1667 - val_accuracy: 0.9813 - val_loss: 0.0810
Epoch 4/5
4695/4695 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - accuracy: 0.9806 - loss: 0.0782 - val_accuracy: 0.9813 - val_loss: 0.0759
Epoch 5/5
4695/4695 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - accuracy: 0.9809 - loss: 0.0741 - val_accuracy: 0.9813 - val_loss: 0.0758
1468/1468 ━━━━━━━━━━━━━━━━━━━━ 1s 626us/step


100%|██████████| 169/169 [1:44:45<00:00, 37.19s/it]


In [23]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Calculate accuracy
accuracy = accuracy_score(y_true, y_pred)
print(f"Accuracy: {accuracy:.4f}")

# Calculate precision
precision = precision_score(y_true, y_pred)
print(f"Precision: {precision:.4f}")

# Calculate recall
recall = recall_score(y_true, y_pred)
print(f"Recall: {recall:.4f}")

# Calculate F1 score
f1 = f1_score(y_true, y_pred)
print(f"F1 Score: {f1:.4f}")


Accuracy: 0.9799
Precision: 0.9875
Recall: 0.9920
F1 Score: 0.9897
